In [266]:
# Notebook criado para o desenvolvimento de envio de ordem do ativo PETR3
#
# Objetivo: Conexao com o MetaTrader 5, e envio de ordem baseados em indicadores
#            exemplo utilizado com o ativo PETR3

In [293]:
# Lendo as bibliotecas necessarias
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from datetime import date
import MetaTrader5 as mt5
import time
# Funcoes para ordem de compra e venda
import sys
sys.path.append('../src/')
from funcoes_compra_e_venda import compra_limitada#, venda_limitada
from funcoes_dos_indicadores import decisao_RSI

In [268]:
# Conectando ao MetaTrader 5
# Utilizando os logins atuais do Software
if not mt5.initialize():
    print("initialize() falhou")
    mt5.shutdown()
# Consultamos o estado e os parâmetros de conexão
print(mt5.terminal_info())
# Obtemos informações sobre a versão do MetaTrader 5
print(mt5.version())

TerminalInfo(community_account=True, community_connection=True, connected=True, dlls_allowed=True, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=True, mqid=True, build=3302, maxbars=100000, codepage=1252, ping_last=239642, community_balance=0.0, retransmission=0.0, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='Portuguese (Brazil)', path='C:\\Program Files\\MetaTrader 5', ...)
(500, 3302, '24 May 2022')


In [269]:
# Volume - Se não for múltiplo de 100, usar o ativo fracionado no simbolo
lote = 0.01 #100
# Simbolo selecionado para grafico
simbolo = "USDSEK" #"PETR3F"

In [270]:
# Obtendo as informacoes do ativo de tamanho 1000, considerando a data de hoje e o Timframe de H1 = 1 hora;
ValoresAtivo = mt5.copy_rates_from_pos(simbolo, mt5.TIMEFRAME_H1, 0, 1000)
ValoresAtivo_PD = pd.DataFrame(ValoresAtivo) # Conversao em pandas
ValoresAtivo_PD['time'] = pd.to_datetime(ValoresAtivo_PD['time'], unit='s')

In [296]:
# Verificando um o tail() dos valores
ValoresAtivo_PD.tail()

,time,open,high,low,close,tick_volume,spread,real_volume
999,2022-05-25 04:00:00,9.76755,9.77769,9.76643,9.77328,1540,949,0


In [289]:
# Processando o Sinal
# Calculando o RSI, com 14 periodos, como parametro de entrada o valor de "close"
close = np.array(ValoresAtivo_PD['close'])
last_element3 = close[-1]
last_price = last_element3

In [273]:

# --------------------------------------------------
# Selecionando o último valor do preço venda
price = mt5.symbol_info_tick(simbolo).ask

In [274]:
# Estudar IC para sugerir como TP e SL

In [295]:
# Encontrando a decisão
decisao = decisao_RSI(close, Periodo = 14)

'NADA'

In [ ]:
# Tomada de decisao e envio de ordem
if decisao == "COMPRAR":
    StopLoss = price - 0.02
    TakeProfit = price + 0.05
    retorno = compra_limitada(simbolo, lote, price, StopLoss, TakeProfit)
    print(retorno)
elif decisao == "VENDER":
    StopLoss = price + 0.02
    TakeProfit = price - 0.05
    retorno = venda_limitada(simbolo, lote, price, StopLoss, TakeProfit)
    print(retorno)
else:
    print('Aguardando o sinal do Mercado')
